## Chain of Responsibility Pattern: NoCodeProgram

- https://github.com/NoCodeProgram/DesignPatterns/blob/main/Behavioral/chainOfResP.ipynb

In [14]:
class Handler:
    def __init__(self):
        self.next_handler = None

    def set_next(self, handler):
        self.next_handler = handler

    def handle(self, request):
        if self.next_handler:
            return self.next_handler.handle(request)
        print("All handlers failed")
        return None


class CashHandler(Handler):
    def handle(self, request):
        if request['method'] == 'cash':
            print(f"processing cash {request['amount']} won")
        else:
            print(f"CashHandler cannot process")
            super().handle(request)


class CreditCardHandler(Handler):
    def handle(self, request):
        if request['method'] == 'credit card':
            print(f"processing creditCard {request['amount']} won")
        else:
            print(f"CreditCardHandler cannot process")
            super().handle(request)


class DebitCardHandler(Handler):
    def handle(self, request):
        if request['method'] == 'debit card':
            print(f"processing debitCard {request['amount']} won")
        else:
            print(f"DebitCardHandler cannot process")
            super().handle(request)

In [15]:
cash_handler = CashHandler()
creditcard_handler = CreditCardHandler()
debicard_handler = DebitCardHandler()

cash_handler.set_next(creditcard_handler)
creditcard_handler.set_next(debicard_handler)

In [16]:
payment = {"method": "cash", "amount": 10000}
cash_handler.handle(payment)

processing cash 10000 won


In [17]:
payment = {"method": "credit card", "amount": 10000}
cash_handler.handle(payment)

CashHandler cannot process
processing creditCard 10000 won


In [19]:
payment = {"method": "debit card", "amount": 10000}
cash_handler.handle(payment)

CashHandler cannot process
CreditCardHandler cannot process
processing debitCard 10000 won


In [8]:
payment = {"method": "paypal", "amount": 10000}
cash_handler.handle(payment)

CashHandler cannot process
CreditCardHandler cannot process
DebitCardHandler cannot process
All handlers failed


## Chain of Responsibility Pattern: Refactoring Guru

- https://refactoring.guru/ko/design-patterns/chain-of-responsibility
- https://refactoring.guru/ko/design-patterns/chain-of-responsibility/python/example

In [21]:
from __future__ import annotations
from abc import ABC, abstractmethod
from typing import Any, Optional

In [30]:
## Handler Interface
class Handler(ABC):
    @abstractmethod
    def set_next(self, handler: Handler) -> Handler:
        pass

    @abstractmethod
    def handle(self, request) -> Optional[str]:
        pass

## Abstract Handler
class AbstractHandler(Handler):
    _next_handler: Handler = None

    def set_next(self, handler: Handler) -> Handler:
        self._next_handler = handler
        return handler

    @abstractmethod
    def handle(self, request: Any) -> str:
        if self._next_handler:
            return self._next_handler.handle(request)
        return None


class MonkeyHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        if request == "Banana":
            return f"Monkey: I'll eat the {request}"
        else:
            return super().handle(request)


class SquirrelHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        if request == "Nut":
            return f"Squirrel: I'll eat the {request}"
        else:
            return super().handle(request)


class DogHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        if request == "MeatBall":
            return f"Dog: I'll eat the {request}"
        else:
            return super().handle(request)

In [31]:
def client_code(handler: Handler) -> None:
    for food in ["Nut", "Banana", "Cup of coffee"]:
        print(f"\nClient: Who wants a {food}?")
        result = handler.handle(food)
        if result:
            print(f"  {result}", end="")
        else:
            print(f"  {food} was left untouched.", end="")


monkey = MonkeyHandler()
squirrel = SquirrelHandler()
dog = DogHandler()

monkey.set_next(squirrel).set_next(dog)

In [32]:
print("Chain: Monkey > Squirrel > Dog")
client_code(monkey)

Chain: Monkey > Squirrel > Dog

Client: Who wants a Nut?
  Squirrel: I'll eat the Nut
Client: Who wants a Banana?
  Monkey: I'll eat the Banana
Client: Who wants a Cup of coffee?
  Cup of coffee was left untouched.

In [33]:
print("Subchain: Squirrel > Dog")
client_code(squirrel)

Subchain: Squirrel > Dog

Client: Who wants a Nut?
  Squirrel: I'll eat the Nut
Client: Who wants a Banana?
  Banana was left untouched.
Client: Who wants a Cup of coffee?
  Cup of coffee was left untouched.

## Chain of Responsibilitydd Pattern: python101.tistory.com

- [[디자인 패턴] 책임 연쇄 패턴 (Chain of Responsibility Pattern) - python 예제 코드](https://python101.tistory.com/entry/%EB%94%94%EC%9E%90%EC%9D%B8-%ED%8C%A8%ED%84%B4-%EC%B1%85%EC%9E%84-%EC%97%B0%EC%87%84-%ED%8C%A8%ED%84%B4-Chain-of-Responsibility-Pattern-python-%EC%98%88%EC%A0%9C-%EC%BD%94%EB%93%9C)

In [36]:
## Abstract Handler
class AbstractLogger:
    DEBUG = 1
    INFO = 2
    ERROR = 3
    
    def __init__(self, level):
        self._level = level
        self._next_logger = None

    def set_next_logger(self, next_logger):
        self._next_logger = next_logger

    def log_message(self, level, message):
        if self._level <= level:
            self.write(message)
        if self._next_logger is not None:
            self._next_logger.log_message(level, message)

    def write(self, message):
        raise NotImplementedError


class ConsoleLogger(AbstractLogger):
    def __init__(self, level):
        super().__init__(level)

    def write(self, message):
        print(f"ConsoleLogger: {message}")


class FileLogger(AbstractLogger):
    def __init__(self, level):
        super().__init__(level)

    def write(self, message):
        print(f"FileLogger: {message}")


class EmailLogger(AbstractLogger):
    def __init__(self, level):
        super().__init__(level)

    def write(self, message):
        print(f"EmailLogger: {message}")

In [38]:
def get_logger_chain():
    error_logger = EmailLogger(AbstractLogger.ERROR)
    file_logger = FileLogger(AbstractLogger.INFO)
    file_logger.set_next_logger(error_logger)
    console_logger = ConsoleLogger(AbstractLogger.DEBUG)
    console_logger.set_next_logger(file_logger)
    return console_logger


logger_chain = get_logger_chain()

logger_chain.log_message(AbstractLogger.ERROR, "An error occurred!")

ConsoleLogger: An error occurred!
FileLogger: An error occurred!
EmailLogger: An error occurred!


In [40]:
logger_chain.log_message(AbstractLogger.INFO, "This is an info message.")

ConsoleLogger: This is an info message.
FileLogger: This is an info message.


In [41]:
logger_chain.log_message(AbstractLogger.DEBUG, "This is a debug message.")

ConsoleLogger: This is a debug message.
